In [1]:
!pip install pygeos
!pip install geopandas
import pandas as pd
import geopandas as gpd

In [2]:
import pygeos
print(f"pygeos version: {pygeos.__version__}")
print(f"geos version: {pygeos.geos_version}")
print(f"use pygeos? {gpd.options.use_pygeos}")

pygeos version: 0.12.0
geos version: (3, 10, 1)
use pygeos? True


In [3]:
import json
import folium
from collections import Counter
from shapely.geometry import LineString, Point
import numpy as np

pd.options.mode.chained_assignment = None

In [4]:
def get_input(path_track, path_graph):
    
    track = pd.read_json(path_track + f'route.json')
    
    with open(path_graph + f'road_graph.json') as f:
        graph = json.load(f)
        f.close()
    
    return track, graph

def get_light(path_to_lights, light_id):
    light = pd.read_csv(path_to_lights + f'light_id_{light_id}.csv')
    return light

def lights_signals(graph, sjoin_nearest_table):

  signals = []
  for i in range(len(sjoin_nearest_table)):
    node_ = sjoin_nearest_table['node_id'][i]

    for u in graph['nodes'][node_]['neighbours']:

      delta = 3
      delta_min, delta_max = u['azimuth'] - delta, u['azimuth'] + delta

      if delta_min <= sjoin_nearest_table['azimuth'][i] <= delta_max:
        signals.append(u['traffic_lights'][0])

  return signals

In [267]:
output, graph = get_input(path_track = '/content/', path_graph = '/content/')
output.head()

,timestamp,latitude,longitude,azimuth,speed
0,2022-06-21 14:33:27+00:00,53.261400,50.240065,47.536787,7.2
1,2022-06-21 14:33:28+00:00,53.261431,50.240120,47.541551,14.4
2,2022-06-21 14:33:29+00:00,53.261474,50.240196,47.549558,21.6
3,2022-06-21 14:33:30+00:00,53.261529,50.240295,47.560935,28.8
4,2022-06-21 14:33:31+00:00,53.261597,50.240415,47.575868,36.0


In [268]:
output['geometry'] = list(zip(output.longitude, output.latitude))
output['geometry'] = output['geometry'].apply(Point)
geo_output = gpd.GeoDataFrame(output[['timestamp', 'azimuth', 'geometry']], geometry = 'geometry', crs='epsg:4326')

# geo_output['sq'] = geo_output.estimate_utm_crs()

geo_output = geo_output.to_crs(32639)
geo_output.head()

,timestamp,azimuth,geometry
0,2022-06-21 14:33:27+00:00,47.536787,POINT (449309.448 5901619.209)
1,2022-06-21 14:33:28+00:00,47.541551,POINT (449313.133 5901622.585)
2,2022-06-21 14:33:29+00:00,47.549558,POINT (449318.300 5901627.314)
3,2022-06-21 14:33:30+00:00,47.560935,POINT (449324.942 5901633.385)
4,2022-06-21 14:33:31+00:00,47.575868,POINT (449333.059 5901640.809)


In [269]:
table = []

for i in graph['nodes']:
    if i['traffic_light']==True:    
      table.append( list((i['longitude'], i['latitude'], i['node_id'])) )

geo_graph = pd.DataFrame(table, columns=['longitude', 'latitude', 'node_id'])

geo_graph['geometry'] = list(zip(geo_graph.longitude, geo_graph.latitude))
geo_graph['geometry'] = geo_graph['geometry'].apply(Point)

geo_graph = gpd.GeoDataFrame(geo_graph[['geometry', 'node_id']], geometry = 'geometry', crs='epsg:4326')
geo_graph = geo_graph.to_crs(32639)
geo_graph

,geometry,node_id
0,POINT (449620.524 5901897.013),3
1,POINT (449389.887 5902170.365),6


In [270]:
nearest = gpd.sjoin_nearest(geo_graph, geo_output, max_distance=7, distance_col='distances')
nearest = nearest.to_crs(epsg=4326)
nearest

,geometry,node_id,index_right,timestamp,azimuth,distances
0,POINT (50.24468 53.26393),3,27,2022-06-21 14:33:54+00:00,49.318017,6.482587
1,POINT (50.24118 53.26636),6,60,2022-06-21 14:34:26+00:00,317.892342,5.110805


In [272]:
signals = lights_signals(graph = graph, sjoin_nearest_table = nearest)
signals

[{'id_traffic_light': 1, 'signal_group': 1},
 {'id_traffic_light': 2, 'signal_group': 2}]

In [273]:
for i in signals[0].keys():
  nearest[i] = np.nan

for i, singl in enumerate(signals):
  sign = list(singl.values())
  nearest['id_traffic_light'][i] = int(sign[0])
  nearest['signal_group'][i] = int(sign[1])

nearest

,geometry,node_id,index_right,timestamp,azimuth,distances,id_traffic_light,signal_group
0,POINT (50.24468 53.26393),3,27,2022-06-21 14:33:54+00:00,49.318017,6.482587,1.0,1.0
1,POINT (50.24118 53.26636),6,60,2022-06-21 14:34:26+00:00,317.892342,5.110805,2.0,2.0


In [274]:
lines = []

for i in zip(nearest['id_traffic_light'], nearest['signal_group']):
  for u in graph['traffic_lights']:

    for group in u['signal_groups']:
      if i[0]==u['id'] and group['id']==i[1]:

        lines.append( group['stop_line'] )

lines

[[{'latitude': 53.263906, 'longitude': 50.24458},
  {'latitude': 53.26389, 'longitude': 50.24462}],
 [{'latitude': 53.266295, 'longitude': 50.24115},
  {'latitude': 53.26631, 'longitude': 50.24118}]]

In [277]:
data_line = pd.DataFrame(columns=['lat_start', 'lng_start', 'lat_end', 'lng_end'])

for i in range(len(lines)):
  data_line.loc[i] = [ lines[i][0]['latitude'], lines[i][0]['longitude'], lines[i][1]['latitude'], lines[i][1]['longitude'] ]

data_line['geom_start'] = list(zip( data_line['lng_start'], data_line['lat_start'] ))
data_line['geom_end'] = list(zip( data_line['lng_end'],data_line['lat_end'] ))

data_line['geometry'] = list(zip( data_line['geom_start'],data_line['geom_end'] ))
data_line['geometry'] = data_line['geometry'].apply(LineString)

geo_data_line = gpd.GeoDataFrame(data_line[['geometry']], geometry='geometry', crs='epsg:4326')
geo_data_line = geo_data_line.to_crs(32639)
geo_data_line

,geometry
0,"LINESTRING (449613.563 5901894.750, 449616.212..."
1,"LINESTRING (449387.607 5902162.935, 449389.625..."


In [288]:
stop_lines_nearest = gpd.sjoin_nearest(geo_data_line, geo_output, max_distance=5, distance_col='distances')
stop_lines_nearest = stop_lines_nearest.to_crs(epsg=4326)
stop_lines_nearest

,geometry,index_right,timestamp,azimuth,distances
0,"LINESTRING (50.24458 53.26391, 50.24462 53.26389)",27,2022-06-21 14:33:54+00:00,49.318017,0.185091
1,"LINESTRING (50.24115 53.26629, 50.24118 53.26631)",60,2022-06-21 14:34:26+00:00,317.892342,4.491202


In [289]:
for i in range(len(stop_lines_nearest)):
  if stop_lines_nearest['distances'][i] >= 5:
    stop_lines_nearest.drop(axis=0, index=i, inplace=True)

In [290]:
near = pd.concat([nearest[['node_id', 'id_traffic_light', 'signal_group']], stop_lines_nearest[['azimuth', 'timestamp', 'index_right', 'distances']]], axis=1)
near

,node_id,id_traffic_light,signal_group,azimuth,timestamp,index_right,distances
0,3,1.0,1.0,49.318017,2022-06-21 14:33:54+00:00,27,0.185091
1,6,2.0,2.0,317.892342,2022-06-21 14:34:26+00:00,60,4.491202


In [291]:
result_table = pd.DataFrame()

for i in range(len(near)):
  
  id = near['node_id'][i]
  signalgroup = int(near['signal_group'][i])
  light_open = get_light(path_to_lights = '/content/', light_id = id)

  for u in range(len(light_open)):
    if pd.to_datetime(light_open['time'][u]) == pd.to_datetime(near['timestamp'][i]):
      
      # check_table = light_open[['time', f'signal_group_{int(signalgroup)}']].iloc[u].to_frame().T.copy()
      check_table = light_open.iloc[u].to_frame().T.copy()

      if light_open[f'signal_group_{int(signalgroup)}'][u]==False:

        check_table['light_id'] = int(id)
        check_table['result'] = 'Red light traffic is not recorded'
        result_table = pd.concat([result_table, check_table])

        display(check_table)
        
      else:

        check_table['light_id'] = int(id)
        check_table['result'] = 'Red light traffic is recorded'
        result_table = pd.concat([result_table, check_table])

        display(check_table)

,time,signal_group_1,signal_group_2,light_id,result
27,2022-06-21 14:33:54+00:00,False,False,1,Red light traffic is not recorded


,time,signal_group_1,signal_group_2,light_id,result
60,2022-06-21 14:34:26+00:00,True,False,2,Red light traffic is not recorded


In [292]:
result_table.drop_duplicates(inplace=True)
result_table.reset_index(drop=False, inplace=True)
result_table.rename(columns={'index':'track_moment'}, inplace=True)
result_table

,track_moment,time,signal_group_1,signal_group_2,light_id,result
0,27,2022-06-21 14:33:54+00:00,False,False,1,Red light traffic is not recorded
1,60,2022-06-21 14:34:26+00:00,True,False,2,Red light traffic is not recorded


In [245]:
my_map = folium.Map(location=[53.261382, 50.240032], tiles='openstreetmap', zoom_start=16)

with open('route.json') as points:
    points = json.load(points)

for i, point in enumerate(points):
  folium.Marker(location=[point['latitude'], point['longitude']], tooltip=f'Метка {i}').add_to(my_map)

with open('road_graph.json') as f:
    test = json.load(f)

    for i in test['nodes']:
        folium.Marker(location=[i['latitude'], i['longitude']],
                      tooltip='node_id ' + str(i['node_id']) +
                      str(i['neighbours']),
                      icon=folium.Icon(color="red")).add_to(my_map)

folium.Choropleth(geo_data=geo_data_line['geometry'][0], line_weight=10, line_color='blue').add_to(my_map)
folium.Choropleth(geo_data=geo_data_line['geometry'][1], line_weight=10, line_color='blue').add_to(my_map)


my_map